# Digital (Binary) Options

This notebook demonstrates cash-or-nothing digital option pricing.

## What is a Digital Option?

A **cash-or-nothing digital** (or binary) option pays a fixed amount if the option expires in-the-money, and nothing otherwise.

- **Digital Call**: Pays `$payout` if $S_T > K$
- **Digital Put**: Pays `$payout` if $S_T < K$

### Key Characteristics

1. **Discontinuous payoff**: Unlike vanilla options, payoff jumps from 0 to payout at the strike
2. **Binary outcome**: Either full payout or nothing
3. **High gamma near strike**: Very sensitive to spot near expiration

### Common Uses

- **Structured products**: As building blocks for complex payoffs
- **Binary options trading**: Popular retail product (though often unregulated)
- **Hedging discrete events**: E.g., earnings announcements
- **Range accruals**: Digital coupons that accrue if spot stays in range

## Setup

In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib.pyplot as plt
import numpy as np

from src.core.option_types import Option, OptionType, ExerciseStyle
from src.models.digital import (
    price_digital_black_scholes,
    price_digital_monte_carlo,
)

# Plot styling
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## Price Digital Options: BS vs Monte Carlo

Let's price digital options at different moneyness levels using both methods.

In [ ]:
# Common parameters
spot = 100.0
rate = 0.05
vol = 0.20
T = 0.5
payout = 100

# Define options at different strikes
strikes = [90, 100, 110]  # ITM, ATM, OTM
labels = ['ITM (K=90)', 'ATM (K=100)', 'OTM (K=110)']

print("Digital Call Options (Payout = $100)")
print("=" * 65)
print(f"{'Strike':<12} {'BS Price':<12} {'MC Price':<12} {'MC SE':<12} {'Diff'}")
print("-" * 65)

for K, label in zip(strikes, labels):
    option = Option(spot, K, rate, vol, T, OptionType.CALL, ExerciseStyle.EUROPEAN)
    
    bs_price = price_digital_black_scholes(option, payout)
    mc_price, mc_se = price_digital_monte_carlo(option, payout, num_paths=100_000, seed=42)
    diff = mc_price - bs_price
    
    print(f"{K:<12} ${bs_price:<11.2f} ${mc_price:<11.2f} ±{mc_se:<10.2f} {diff:+.2f}")

print()
print("Digital Put Options (Payout = $100)")
print("=" * 65)
print(f"{'Strike':<12} {'BS Price':<12} {'MC Price':<12} {'MC SE':<12} {'Diff'}")
print("-" * 65)

for K, label in zip(strikes, labels):
    option = Option(spot, K, rate, vol, T, OptionType.PUT, ExerciseStyle.EUROPEAN)
    
    bs_price = price_digital_black_scholes(option, payout)
    mc_price, mc_se = price_digital_monte_carlo(option, payout, num_paths=100_000, seed=42)
    diff = mc_price - bs_price
    
    print(f"{K:<12} ${bs_price:<11.2f} ${mc_price:<11.2f} ±{mc_se:<10.2f} {diff:+.2f}")

### Observations

- **ITM calls (K=90)**: High price, close to discounted payout
- **ATM options (K=100)**: Price ≈ 0.5 × discounted payout
- **OTM calls (K=110)**: Low price
- **Call + Put = Discounted Payout**: Always one pays, so sum = 100 × e^(-rT)

## Digital Call Price vs Strike

The characteristic "kink" profile shows the step-function nature of digital payoffs.

In [ ]:
# Generate price curve across strikes
strikes_range = np.linspace(70, 130, 50)
call_prices = []
put_prices = []

for K in strikes_range:
    call_opt = Option(spot, K, rate, vol, T, OptionType.CALL, ExerciseStyle.EUROPEAN)
    put_opt = Option(spot, K, rate, vol, T, OptionType.PUT, ExerciseStyle.EUROPEAN)
    
    call_prices.append(price_digital_black_scholes(call_opt, payout))
    put_prices.append(price_digital_black_scholes(put_opt, payout))

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(strikes_range, call_prices, 'b-', linewidth=2, label='Digital Call')
ax.plot(strikes_range, put_prices, 'r--', linewidth=2, label='Digital Put')

# Mark ATM
ax.axvline(x=spot, color='gray', linestyle=':', alpha=0.7, label='ATM')

# Mark max price (discounted payout)
max_price = payout * np.exp(-rate * T)
ax.axhline(y=max_price, color='green', linestyle=':', alpha=0.5, 
           label=f'Max Price (${max_price:.2f})')

ax.set_xlabel('Strike Price ($)', fontsize=12)
ax.set_ylabel('Option Price ($)', fontsize=12)
ax.set_title('Digital Option Price vs Strike', fontsize=14)
ax.legend(loc='center right', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### The "Kink" Profile

- Digital call price decreases steeply near ATM (high sensitivity)
- Digital put is the mirror image
- Both approach their limits (max or 0) as they go deeper ITM/OTM
- The steepness near ATM reflects the large gamma (second derivative)

## Effect of Volatility on Digital Options

Unlike vanilla options, ATM digitals can **decrease** in value with higher volatility!

In [ ]:
# Vary volatility
vols = [0.10, 0.20, 0.30, 0.40, 0.50]

print("Digital Call Price vs Volatility (ATM, K=100)")
print("=" * 40)
print(f"{'Volatility':<15} {'Price'}")
print("-" * 40)

for v in vols:
    option = Option(spot, 100, rate, v, T, OptionType.CALL, ExerciseStyle.EUROPEAN)
    price = price_digital_black_scholes(option, payout)
    print(f"{v:.0%}            ${price:.2f}")

print()
print("Note: Higher vol spreads out the distribution, reducing")
print("the probability of ending just above the strike.")

In [ ]:
# Visualize vol effect at different strikes
vols_range = np.linspace(0.05, 0.60, 30)
strikes_test = [90, 100, 110]

fig, ax = plt.subplots(figsize=(10, 6))

for K in strikes_test:
    prices = []
    for v in vols_range:
        opt = Option(spot, K, rate, v, T, OptionType.CALL, ExerciseStyle.EUROPEAN)
        prices.append(price_digital_black_scholes(opt, payout))
    
    label = f'K={K} ({"ITM" if K < spot else "ATM" if K == spot else "OTM"})'
    ax.plot(vols_range * 100, prices, linewidth=2, label=label)

ax.set_xlabel('Volatility (%)', fontsize=12)
ax.set_ylabel('Digital Call Price ($)', fontsize=12)
ax.set_title('Digital Call Price vs Volatility', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Key Insight: Negative Vega for OTM Digitals

- **ITM digitals**: Positive vega (higher vol can knock them OTM)
- **ATM digitals**: Vega ≈ 0 (symmetric effect)
- **OTM digitals**: Negative vega (higher vol can bring them ITM)

This is opposite to vanilla options, which always have positive vega!

## Summary

| Property | Digital Call | Digital Put |
|----------|--------------|-------------|
| Payoff | Fixed if S_T > K | Fixed if S_T < K |
| Price Range | [0, payout × e^(-rT)] | [0, payout × e^(-rT)] |
| ATM Price | ≈ 0.5 × discounted payout | ≈ 0.5 × discounted payout |
| Delta | Positive (can be very large) | Negative (can be very large) |
| Vega | Can be negative (OTM) | Can be negative (OTM) |

### Trading Considerations

1. **Pin risk**: Near expiration at-the-money, small moves cause large P&L
2. **Hedging**: Requires frequent rebalancing due to high gamma
3. **Skew sensitivity**: More sensitive to vol smile than vanillas
4. **Bid-ask spreads**: Usually wider due to hedging difficulty